In [1]:
import math
import pandas as pd
import numpy as np

In [2]:
def load_dataset(path):
    df = pd.read_csv(path, delimiter=',' , engine='python')
    return df

In [3]:
path="..\DatasetExos.csv"
df = load_dataset(path)

In [22]:
df.loc[df["Category"] == "heav", "Category"] = "heavy"
df.loc[df["Exercise"] == "reste", "Exercise"] = "rest"
df.loc[df["Exercise"] == "raw", "Exercise"] = "row"
df= df.dropna()

In [ ]:
def manhattan_distance(instance1,instance2,columns):
    d = 0

    for column in columns:
        d +=  abs(instance1[column] - instance2[column])
    
    return d

In [5]:
def hamming_distance(instance1, instance2, columns):
    d = 0

    for column in columns:
       
        d += sum(c1 != c2 for c1, c2 in zip(instance1[column], instance2[column]))
    
    return d


In [6]:
def instance_distance(instance1, instance2, numerical_columns, categorical_columns):

    d1 = manhattan_distance(instance1,instance2,numerical_columns) 
    d2 = hamming_distance(instance1, instance2, categorical_columns)
    return d1 + d2

In [24]:
numerical_cols = df.select_dtypes(include=[np.number]).columns
non_numerical_cols = df.select_dtypes(exclude=[np.number]).columns
non_numerical_cols = non_numerical_cols.difference(['Exercise'])
print(numerical_cols)
print(non_numerical_cols)

Index(['Acc_x', 'Acc_y', 'Acc_z', 'Gyro_x', 'Gyro_y', 'Gyro_z', 'Set'], dtype='object')
Index(['Category', 'ID', 'ep (ms)'], dtype='object')


In [25]:
instance1 = df.iloc[0]  # Get the first row
instance2 = df.iloc[1]

In [26]:
d = instance_distance(instance1, instance2, numerical_cols, non_numerical_cols)
d

5.7994

In [31]:
def sorting_distances(df, instance, numerical_columns, categorical_columns):
    distances = []
    
    for _, row in df.iterrows():  
        d = instance_distance(instance.iloc[0], row, numerical_columns, categorical_columns)
        distances.append((row, d))  
    
    distances = sorted(distances, key=lambda x: x[1])  
    return distances

In [15]:
def dominant_class(top_k_distances):
    classes = {}

    for row, distance in top_k_distances:
        if 'Exercise' in row:
            if row['Exercise'] in classes:
                classes[row['Exercise']] += 1
            else:
                classes[row['Exercise']] = 1
        else:
            raise KeyError("Column 'Exercise' not found in the row data.")

   
    sorted_classes = sorted(classes.items(), key=lambda x: x[1], reverse=True)

   
    return sorted_classes[0][0] if sorted_classes else None


In [16]:
def knn(df,instance,k,numerical_columns, categorical_columns):

    sorted_distances = sorting_distances(df, instance, numerical_columns, categorical_columns)
 
    top_k_distances = sorted_distances[:k]

    return dominant_class(top_k_distances)

In [27]:
X = {
    'ep (ms)': '2024-11-20 18:09:51.000',  
    'Acc_x': -0.137,
    'Acc_y': 1.066,
    'Acc_z': 0.8215,
    'Gyro_x': -6.597,
    'Gyro_y': 0.808,
    'Gyro_z': 1.985,
    'ID': 'B',
    'Category': 'medium',
    'Set': 30
}

instance_df = pd.DataFrame([X])
instance_df

,ep (ms),Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,ID,Category,Set
0,2024-11-20 18:09:51.000,-0.137,1.066,0.8215,-6.597,0.808,1.985,B,medium,30


In [33]:
predicted_class = knn(df, instance_df, 3, numerical_cols, non_numerical_cols)
print(f'The predicted class for the instance is {predicted_class}')

The predicted class for the instance is bench


In [35]:
predicted_class = knn(df, instance_df, 10, numerical_cols, non_numerical_cols)
print(f'the predicted class for the instance is {predicted_class}')

the predicted class for the instance is squat
